In [35]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import gensim.downloader as api
import gensim
import operator
from nltk.util import ngrams 
import nltk
from nltk.corpus import stopwords
from nltk import download
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from gensim import corpora

import csv
from textstat.textstat import textstat
stop_words = stopwords.words('english')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/EthanHsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [167]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]

### Fields in instances.jsonl:
##{
#"id" : "<instance id>",
#"postTimestamp" : "<weekday> <month> <day> <hour>:<minute>:<second><time_offset> <year>",
#"postText" : ["<text of the tweet post with links removed>"],
#"postMedia" : ["<path to a file in the media archive>"],
#"targetTitle" : "<title of target news article>",
#"targetDescription": "<description tag of target news article>",
#"targetKeywords" : "<keywords tag of target news article>",
#"targetParagraphs" : ["<text of the i-th paragraph in the target article>"],
#"targetCaptions" : ["<caption of the i-th image in the target article>"]}
### Fields in truth.jsonl:
#{
#"id" : "<instance id>",
#"truthJudgments" : [<number in [0,1]>],
#"truthMean" : <number in [0,1]>,
#"truthMedian" : <number in [0,1]>,
#"truthMode" : <number in [0,1]>,
#truthClass" : "clickbait | no-clickbait"
#}

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [168]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'id': '7',
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'truthMean': 0.2,
 'truthMedian': 0.33333334,
 'truthMode': 0.33333334}

In [169]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [170]:
raw_instance_train = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [171]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        non_clickbait_train.append(raw_instance_train[x])

In [172]:
truth_label = []
for x in range(0, len(raw_instance_train)):
    truth_label.append(raw_truth_train[x]['truthClass'])
len(truth_label)

17581

In [173]:
#clickbait_train
clickbait_train[0]

{'id': '2',
 'postMedia': [],
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'targetKeywords': '',
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns 

In [174]:
#non_clickbait_train
non_clickbait_train[0]

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [175]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle']))
targetTitleCountWords

[17,
 8,
 10,
 11,
 6,
 11,
 10,
 11,
 12,
 14,
 10,
 12,
 12,
 23,
 10,
 14,
 9,
 24,
 7,
 10,
 12,
 10,
 7,
 5,
 11,
 14,
 8,
 11,
 10,
 7,
 14,
 8,
 11,
 10,
 9,
 11,
 7,
 19,
 11,
 1,
 9,
 13,
 12,
 3,
 10,
 299,
 15,
 9,
 10,
 295,
 11,
 21,
 7,
 7,
 10,
 11,
 10,
 9,
 10,
 10,
 12,
 8,
 19,
 10,
 307,
 295,
 8,
 4,
 5,
 9,
 9,
 4,
 11,
 10,
 276,
 9,
 21,
 9,
 8,
 8,
 11,
 8,
 18,
 6,
 20,
 8,
 12,
 15,
 10,
 1,
 12,
 9,
 20,
 10,
 8,
 13,
 13,
 11,
 13,
 6,
 12,
 12,
 10,
 11,
 12,
 10,
 8,
 11,
 12,
 10,
 10,
 11,
 11,
 9,
 8,
 10,
 14,
 7,
 8,
 14,
 10,
 8,
 20,
 15,
 11,
 10,
 6,
 14,
 9,
 12,
 10,
 9,
 13,
 17,
 7,
 10,
 12,
 6,
 18,
 17,
 8,
 7,
 10,
 13,
 16,
 17,
 12,
 13,
 301,
 7,
 4,
 11,
 12,
 10,
 12,
 18,
 9,
 12,
 7,
 5,
 12,
 4,
 8,
 11,
 9,
 9,
 9,
 13,
 13,
 2,
 10,
 8,
 7,
 11,
 10,
 10,
 20,
 10,
 10,
 9,
 7,
 10,
 11,
 25,
 11,
 36,
 11,
 12,
 3,
 8,
 7,
 9,
 9,
 18,
 303,
 11,
 16,
 12,
 11,
 9,
 7,
 10,
 13,
 5,
 10,
 10,
 7,
 20,
 11,
 12,
 6,
 9,
 7,
 8,


In [176]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))


In [177]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


12.12

In [178]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [179]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [180]:
#clickbait_train, emoji
def pos_1gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
#for single_data in clickbait_train:
    #print(pos_2gram(single_data, "postText"))

In [181]:
pos_1gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [182]:
#calculate tags in a data
def cal_pos_1grams_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        #print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [183]:
cal_pos_1grams_tag(clickbait_train[0], "postText", "NNP")
cal_pos_1grams_tag_list = []
for x in range(0, len(raw_instance_train)):
    cal_pos_1grams_tag_list.append(cal_pos_1grams_tag(raw_instance_train[x], "postText", "NNP"))
cal_pos_1grams_tag_list

[1,
 3,
 5,
 2,
 0,
 0,
 2,
 0,
 4,
 4,
 1,
 1,
 2,
 4,
 1,
 9,
 4,
 0,
 0,
 3,
 4,
 0,
 3,
 4,
 4,
 0,
 5,
 1,
 1,
 1,
 1,
 0,
 0,
 5,
 1,
 2,
 0,
 10,
 3,
 2,
 1,
 7,
 2,
 2,
 3,
 1,
 2,
 2,
 4,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 0,
 3,
 2,
 4,
 7,
 0,
 0,
 1,
 8,
 2,
 10,
 0,
 4,
 4,
 0,
 3,
 1,
 1,
 1,
 5,
 6,
 1,
 1,
 5,
 0,
 3,
 1,
 1,
 4,
 5,
 3,
 3,
 2,
 4,
 1,
 3,
 4,
 3,
 6,
 3,
 2,
 4,
 1,
 2,
 1,
 3,
 6,
 4,
 2,
 2,
 3,
 8,
 3,
 11,
 4,
 2,
 5,
 1,
 1,
 7,
 4,
 4,
 1,
 1,
 3,
 0,
 1,
 8,
 3,
 0,
 2,
 2,
 3,
 2,
 6,
 3,
 1,
 3,
 2,
 2,
 5,
 8,
 5,
 1,
 4,
 2,
 6,
 0,
 1,
 2,
 1,
 6,
 3,
 1,
 3,
 4,
 2,
 1,
 3,
 2,
 3,
 2,
 5,
 8,
 1,
 0,
 7,
 2,
 4,
 3,
 4,
 3,
 7,
 13,
 5,
 0,
 3,
 6,
 1,
 2,
 4,
 2,
 2,
 5,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 8,
 2,
 3,
 2,
 7,
 2,
 2,
 4,
 2,
 1,
 9,
 2,
 1,
 3,
 2,
 3,
 4,
 0,
 3,
 1,
 0,
 2,
 0,
 5,
 7,
 4,
 2,
 0,
 0,
 4,
 0,
 5,
 2,
 3,
 3,
 2,
 3,
 10,
 2,
 7,
 5,
 3,
 3,
 8,
 3,
 7,
 3,
 8,
 3,
 2,
 1,
 2,
 0,
 2,
 5,
 1,
 0,
 0,
 4,
 0,
 0,

In [184]:
#yutarochan's code
def pos_2gram(title, p1, p2):
    pos_list = pos_tag(word_tokenize(title))
    return sum(map(lambda x: x[0][1] == p1 and x[1][1] == p2, zip(pos_list[:-1], pos_list[1:])))

In [185]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [186]:
pos_2gram(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [187]:
#revised pos_2grams
def pos_2gram_sum(title, p1, p2):
    pos_2g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2g_num += 1
    return pos_2g_num

In [188]:
pos_2gram_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP'))
pos_2gram_NNP_NNP

[0,
 2,
 2,
 1,
 0,
 2,
 5,
 0,
 3,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 4,
 4,
 3,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 8,
 3,
 0,
 0,
 32,
 0,
 3,
 3,
 30,
 0,
 0,
 0,
 3,
 0,
 4,
 6,
 0,
 4,
 0,
 6,
 3,
 0,
 5,
 32,
 34,
 5,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 77,
 2,
 2,
 5,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 5,
 2,
 0,
 7,
 3,
 1,
 1,
 1,
 0,
 4,
 2,
 4,
 0,
 0,
 4,
 7,
 0,
 1,
 1,
 4,
 3,
 6,
 6,
 1,
 0,
 2,
 4,
 0,
 1,
 9,
 1,
 0,
 3,
 7,
 0,
 0,
 3,
 1,
 5,
 0,
 5,
 6,
 4,
 4,
 1,
 2,
 0,
 0,
 5,
 5,
 1,
 3,
 0,
 1,
 2,
 7,
 7,
 7,
 2,
 0,
 3,
 28,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 1,
 4,
 1,
 0,
 3,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 4,
 3,
 1,
 0,
 3,
 3,
 4,
 2,
 1,
 9,
 1,
 0,
 0,
 4,
 2,
 0,
 5,
 2,
 0,
 3,
 6,
 34,
 1,
 0,
 0,
 4,
 5,
 4,
 3,
 3,
 1,
 4,
 0,
 4,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 5,
 0,
 0,
 5,
 1,
 44,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 4,
 1,
 0,
 0,
 4,
 0,
 2,
 3,
 6,
 0,
 3,
 31,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 1,

In [189]:
#check pos_2grams
def pos_2gram_check(title, p1, p2):
    pos_2gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2gram_list.append([pos_z[0], pos_z[1]])
    return pos_2gram_list

In [190]:
print(pos_2gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP'))

[[('U.S.', 'NNP'), ('Soccer', 'NNP')], [('Hope', 'NNP'), ('Solo', 'NNP')]]


In [191]:
#revised pos_3grams
def pos_3gram_sum(title, p1, p2, p3):
    pos_3g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3g_num += 1
    return pos_3g_num

In [192]:
pos_3gram_NNP_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'NNP'))
len(pos_3gram_NNP_NNP_NNP)
print(pos_3gram_NNP_NNP_NNP)

[0, 1, 0, 0, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 12, 0, 1, 2, 9, 0, 0, 0, 1, 0, 2, 4, 0, 3, 0, 2, 2, 0, 3, 9, 14, 4, 0, 1, 0, 0, 0, 0, 1, 49, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 5, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 3, 4, 0, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 0, 0, 5, 0, 0, 1, 5, 0, 0, 1, 0, 4, 0, 2, 4, 2, 0, 0, 1, 0, 0, 3, 1, 0, 1, 0, 0, 0, 4, 4, 3, 1, 0, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 2, 1, 1, 0, 6, 0, 0, 0, 3, 1, 0, 3, 1, 0, 2, 3, 12, 0, 0, 0, 1, 3, 3, 2, 2, 0, 3, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 26, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 3, 0, 1, 12, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 7, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 3, 1, 1, 0, 4, 6, 2, 0, 0, 0, 0, 3, 0, 0, 16, 2, 1, 0, 0, 6, 0, 0, 0, 0, 1, 0, 0, 6, 2, 27, 0, 2, 0, 3, 0, 0, 1, 0, 0, 0, 0

In [193]:
#check pos_3grams
def pos_3gram_check(title, p1, p2, p3):
    pos_3gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3gram_list.append([pos_z[0], pos_z[1],pos_z[2]])
    return pos_3gram_list

In [194]:
print(pos_3gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

[]


In [195]:
target_para_list = []
for n in clickbait_train[0]['targetParagraphs']:
    target_para_ = doc_punc_parser(n)
    #print(type(target_para_list))
    #print(target_para_)
    target_para_list = target_para_list + target_para_
    #print(n)
    #print(nltk.pos_tag(n))

target_para_tags = nltk.pos_tag(target_para_list)

In [196]:
def count_tags (tags_list):
    tag_fd = nltk.FreqDist(tag for (word, tag) in tags_list)
    return (tag_fd.most_common())

In [197]:
print(count_tags(target_para_tags))

#for x in range(0, len(raw_instance_train)):


[('NN', 115), ('NNP', 90), ('DT', 88), ('IN', 87), ('JJ', 44), ('VBZ', 41), ('RB', 40), ('.', 31), ('VB', 30), (',', 27), ('NNS', 24), ('TO', 23), ('CC', 20), ('POS', 16), ('VBD', 15), ('VBG', 14), ('PRP', 13), ('VBN', 12), ('PRP$', 10), ("''", 10), ('``', 9), ('VBP', 6), ('MD', 6), ('JJR', 5), ('WP', 4), ('WRB', 3), (':', 3), ('CD', 3), ('EX', 3), ('JJS', 2), ('(', 2), ('WDT', 2), (')', 2), ('RBR', 2), ('RBS', 1), ('RP', 1), ('NNPS', 1)]


In [198]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].most_common(5)) for tag in cfd.conditions())

In [199]:
findtags("NN",target_para_tags)

{'NN': [('violence', 6),
  ('lot', 4),
  ('incident', 3),
  ('explanation', 3),
  ('game', 3)],
 'NNP': [('Solo', 15),
  ('U.S.', 11),
  ('Soccer', 10),
  ('Gulati', 5),
  ('Hope', 4)],
 'NNPS': [('Americans', 1)],
 'NNS': [('women', 2),
  ('questions', 2),
  ('months', 2),
  ('concerns', 1),
  ('values', 1)]}

In [222]:
###Oct 6, Ethan's new features
#Some methods I need
#assorted cleaning
stopWords = stopwords.words('english')

def stem_tokens(tokens, ps):
    stemmed = []
    for item in tokens:
        stemmed.append(ps.stem(item))
    return stemmed
ps = PorterStemmer()

def lemmatize_tokens(tokens, wnl):
    lemmatized = []
    for item in tokens:
        lemmatized.append(wnl.lemmatize(item))
    return lemmatized
wnl = WordNetLemmatizer()

def tokensClean(text):
    tokens = word_tokenize(text)
    tokens = stem_tokens(tokens, ps)
    tokens = lemmatize_tokens(tokens, wnl)
    tokens = [w for w in tokens if not w in stopWords]
    tokens = [c for c in tokens if c not in punctuation]
    return (tokens)

def doc_punc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    #clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return non_empty_tokens

def tweet_Tokenizer(document):
    tknzr = TweetTokenizer()
    #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    
    return [c for c in tknzr.tokenize(document) if c not in punctuation]

In [251]:
##1. Number of Characters. 
def count_array_char(text):
    if text != []:
        tokens = [doc_punc_parser(t) for t in text]
        return sum([sum([len(t) for t in token])for token in tokens])
    else:
        return -1
    
def count_string_char(text):
    if text != "":
        tokens = doc_punc_parser(text)
        return sum([len(t) for t in tokens])
    else:
        return -1

def count_tweet_array_char(text):
    if text != []:
        tokens = [tweet_Tokenizer(t) for t in text]
        return sum([sum([len(t) for t in token])for token in tokens])
    else:
        return -1
    
def count_tweet_string_char(text):
    if text != "":
        tokens = tweet_Tokenizer(text)
        return sum([len(t) for t in tokens])
    else:
        return -1

In [252]:
##2. Number of Words. (include punctuation)
def count_array_word(text):
    tokens = [doc_punc_parser(t) for t in text]
    return sum([len(token) for token in tokens])
def count_string_word(text):
    token = doc_punc_parser(text)
    return (len(token))

#3. Numbers of words(exclude punctuation, and '@'/'#')
def count_tweet_array_word(text):
    if text != []:
        tokens = [tweet_Tokenizer(t) for t in text]
        return sum([len(token) for token in tokens])
    else:
        return -1
def count_tweet_string_word(text):
    if text != "":
        token = tweet_Tokenizer(text)
        return len(token)
    else:
        return -1

In [253]:
##3. Common Words Between Article Keywords and Others.
#parsing the targetKeywords.
#Made a new function for parsing since targetKeywords are separated by "," 
def keyWord_parse_(keywords):
    keyList = keywords.split(",")
    #keyList = stem_tokens(keyList, ps)
    #keyList = lemmatize_tokens(keyList, wnl)  
    if (keyList) != ['']:
        return keyList
    else:
        #print("keyList",keyList)
        return -1
    
import re
def keyWordFrequency_string_(keyList, strings):
    counter = 0
    strings = strings.lower()
    if keyList != -1 and not (strings.isspace()) and strings != '':
        for word in keyList:
            pattern = re.compile(r'\b{}\b'.format(word), re.I) # case insensitive
            num = len(pattern.findall(strings))
            counter += num
        if count_string_word(strings) == 0:
            print("11:",strings,",",len(strings))
        return counter/count_string_word(strings)
    else:
        return -1

def keyWordFrequency_array_(keyList, array):
    counter = 0
    if keyList != -1 and not (all('' == s or s.isspace() for s in array)) and array != []:
        for strings in array:
            for word in keyList:
                pattern = re.compile(r'\b{}\b'.format(word), re.I) # case insensitive
                if pattern.findall(strings):
                    num = len(pattern.findall(strings))
                    counter += num
        if count_array_word(array) == 0:
            print('array:',array)
        return counter/count_array_word(array)
    else:
        return -1

In [254]:
##4.Formal, and Informal English Word Ratio in postText
def formal_word_preprocess(text):
    if text != [] and not (all('' == s or s.isspace() for s in text)):
        stoplist = list('for a of the and to in'.split())
        formal_list = lemmatize_tokens(tweet_Tokenizer(text), wnl)
        formal_list = [f.replace("'s", "") for f in formal_list]
        formal_list = list(filter((lambda x: x not in punctuation and x not in stoplist), formal_list))
        return formal_list
    else:
        return -1

def count_formal_ratio(text):
    text = text[0]
    counter = 0
    formal_list = formal_word_preprocess(text)
    if formal_list != -1:
        for f in formal_list:
            definitions = wn.synsets(f)
            if len(definitions) > 0:
                counter += 1
        formal_ratio = counter / len(formal_word_preprocess(text))
        #print(formal_word_preprocess(text))
        return formal_ratio
    else:
        return -1

In [255]:
#5. clickbait has more '#' and '@'
def paragraphs_tags_parse(paragraphs):
    if len(paragraphs) != 0:
        return [doc_punc_parser(para) for para in paragraphs]
    else:
        return -1
def tags_parse(tagwords):
    tagList = tagwords.split(",")
    if tagList != ['']:
        return tagList
    else:
        print("keyList",tagList)
        return -1
def tagFrequency_array(keyList, array):  
    if keyList != -1 and array != -1:
        counter = 0
        for key in keyList:
            for strings in array:
                for s in strings:
                    if s == key:
                        #print(strings, key)
                        counter += 1
        return counter
    else:
        return -1

In [256]:
#1a. the number of characters in post’s title.
postTitle_char_count = []
for x in range(0, len(raw_instance_train)):
    postTitle_char_count.append(count_array_char(raw_instance_train[x]['postText']))
print(postTitle_char_count)

[63, 87, 78, 52, 76, 54, 77, 57, 70, 67, 62, 55, 68, 73, 58, 121, 59, 58, 42, 83, 74, 38, 70, 48, 54, 54, 57, 43, 57, 52, 76, 21, 95, 99, 44, 53, 45, 98, 93, 96, 41, 80, 54, 79, 62, 77, 67, 44, 94, 73, 52, 76, 66, 49, 44, 58, 48, 67, 62, 100, 70, 37, 61, 64, 76, 80, 65, 75, 89, 84, 70, 29, 72, 67, 97, 43, 43, 55, 33, 60, 42, 52, 42, 95, 68, 80, 60, 88, 47, 41, 72, 40, 78, 50, 53, 59, 54, 53, 77, 71, 54, 78, 85, 69, 57, 54, 91, 76, 51, 68, 119, 55, 58, 47, 52, 87, 72, 80, 47, 70, 69, 48, 74, 56, 72, 45, 42, 59, 68, 67, 125, 71, 81, 71, 44, 79, 65, 120, 96, 47, 81, 63, 82, 62, 73, 78, 80, 72, 79, 82, 64, 69, 61, 74, 78, 75, 60, 68, 68, 76, 70, 56, 86, 51, 97, 99, 87, 65, 81, 98, 61, 60, 95, 72, 71, 64, 71, 41, 53, 66, 39, 107, 73, 88, 56, 89, 78, 60, 71, 53, 24, 51, 65, 74, 73, 77, 99, 51, 85, 46, 35, 71, 71, 60, 77, 50, 83, 73, 52, 61, 26, 99, 97, 57, 69, 74, 65, 55, 35, 72, 70, 63, 118, 46, 61, 101, 59, 61, 75, 74, 85, 60, 56, 56, 96, 75, 66, 66, 69, 67, 64, 100, 54, 61, 35, 64, 100, 6

In [257]:
#1b. the number of characters in article’s title.
targetTitle_char_count = []
for x in range(0, len(raw_instance_train)):
    targetTitle_char_count.append(count_string_char(raw_instance_train[x]['targetTitle']))
print(targetTitle_char_count)

[87, 47, 57, 52, 29, 54, 67, 56, 52, 65, 62, 62, 66, 131, 57, 57, 59, 118, 42, 47, 64, 48, 47, 29, 58, 51, 55, 44, 38, 34, 64, 34, 67, 43, 53, 53, 43, 94, 53, 4, 41, 66, 54, 25, 62, 1631, 83, 44, 51, 1592, 52, 136, 48, 49, 44, 58, 46, 45, 62, 55, 62, 37, 98, 40, 1655, 1600, 53, 12, 22, 41, 51, 21, 73, 48, 1490, 43, 119, 55, 33, 32, 73, 39, 90, 37, 119, 37, 60, 85, 40, 6, 72, 40, 103, 50, 43, 63, 67, 52, 61, 34, 62, 50, 59, 59, 57, 52, 38, 47, 50, 55, 67, 55, 57, 48, 52, 62, 87, 29, 57, 60, 55, 50, 89, 79, 54, 45, 41, 66, 68, 50, 52, 57, 72, 96, 44, 58, 62, 51, 96, 91, 47, 39, 66, 77, 71, 73, 47, 72, 1620, 33, 22, 58, 61, 61, 68, 89, 43, 66, 36, 35, 69, 29, 44, 51, 56, 48, 41, 65, 65, 12, 61, 57, 30, 58, 36, 64, 82, 40, 53, 41, 39, 62, 66, 127, 56, 165, 64, 60, 20, 53, 24, 51, 43, 85, 1634, 58, 75, 51, 55, 61, 35, 50, 65, 38, 63, 49, 56, 96, 44, 60, 26, 56, 38, 34, 85, 73, 54, 53, 33, 1645, 54, 38, 90, 36, 40, 64, 99, 57, 49, 17, 37, 37, 43, 118, 44, 64, 56, 64, 1633, 76, 61, 100, 54, 6

In [258]:
#1c. the number of characters in article’s description.
targetDescription_char_count = []
for x in range(0, len(raw_instance_train)):
    targetDescription_char_count.append(count_string_char(raw_instance_train[x]['targetDescription']))
print(targetDescription_char_count)

[154, 140, 132, 100, 103, -1, 123, 143, 115, 135, 113, 134, 31, 159, 61, 83, 110, 162, 111, 109, 68, 106, 38, 74, 123, -1, 148, -1, -1, 92, 124, 90, 119, 136, -1, 103, 127, 145, 248, 133, 103, 145, 119, -1, 218, 86, 104, 147, 118, 30, 166, 153, 100, 158, 141, 113, 108, 73, 92, 195, 118, 28, 125, 257, 124, 69, 109, 76, 92, 51, 211, 114, 126, 152, 70, 85, 146, 134, 110, 126, -1, 173, 156, 125, 136, 105, 93, 139, 143, -1, 100, 87, 146, 114, 50, 397, 113, 256, 176, 131, 189, 240, -1, 175, 78, 81, 214, 93, -1, 256, 129, 118, 131, 299, 127, 121, -1, 25, 189, -1, 110, 30, 112, 131, 66, -1, 44, -1, 187, -1, 160, 135, 162, 96, 103, 102, 153, 114, 121, 190, 131, 126, 134, 367, 134, 95, 73, 138, 65, 111, 70, 129, -1, 117, 129, 182, 209, 31, 171, -1, 227, 131, 118, 219, 122, 104, 142, 122, -1, -1, 107, 99, 150, 181, 47, 109, 145, 124, 116, 113, -1, 46, 71, 149, 142, 160, 160, 123, 75, 130, 133, 190, -1, 74, 84, 74, 185, 83, 105, 125, 47, 131, 157, 133, 160, 115, 105, 150, 121, 133, 64, 128, 153, 1

In [259]:
#1d. the number of characters in article’s keywords.
targetKeywords_char_count = []
for x in range(0, len(raw_instance_train)):
    targetKeywords_char_count.append(count_tweet_array_char(raw_instance_train[x]['targetKeywords']))
print(targetKeywords_char_count)

[66, 111, 0, 106, 54, 0, 52, 0, 142, 0, 41, 33, 94, 14, 72, 15, 47, 80, 47, 0, 37, 0, 0, 56, 210, 0, 165, 0, 0, 0, 0, 0, 99, 45, 0, 53, 168, 26, 0, 0, 59, 25, 0, 0, 0, 89, 0, 134, 77, 87, 0, 116, 0, 290, 12, 0, 0, 63, 0, 72, 50, 65, 55, 0, 199, 95, 94, 38, 25, 84, 0, 133, 68, 0, 71, 84, 98, 133, 72, 45, 94, 0, 76, 75, 45, 0, 123, 107, 4, 0, 0, 0, 71, 85, 0, 19, 0, 0, 314, 72, 16, 0, 0, 0, 34, 0, 137, 25, 0, 0, 56, 26, 58, 0, 61, 46, 0, 0, 37, 0, 0, 28, 0, 0, 80, 0, 52, 0, 15, 0, 135, 94, 70, 14, 99, 35, 26, 0, 18, 14, 61, 116, 88, 0, 99, 42, 83, 0, 83, 46, 22, 83, 0, 63, 191, 66, 0, 94, 0, 0, 0, 190, 0, 0, 0, 0, 115, 51, 70, 0, 70, 89, 0, 0, 0, 109, 44, 0, 0, 101, 0, 37, 110, 109, 0, 27, 56, 98, 23, 61, 96, 0, 0, 13, 69, 38, 0, 74, 0, 0, 135, 305, 63, 4, 83, 33, 0, 56, 140, 79, 41, 0, 104, 0, 107, 0, 0, 119, 150, 79, 78, 0, 41, 120, 40, 0, 86, 89, 49, 0, 39, 0, 60, 96, 134, 15, 0, 124, 95, 30, 55, 0, 0, 112, 42, 103, 0, 141, 20, 48, 0, 67, 0, 17, 0, 53, 46, 49, 30, 0, 0, 57, 82, 120, 2

In [260]:
#1e. the number of characters in article’s captions.
targetCaptions_char_count = []
for x in range(0, len(raw_instance_train)):
    targetCaptions_char_count.append(count_array_char(raw_instance_train[x]['targetCaptions']))
print(targetCaptions_char_count)

[2330, 1879, 362, 875, 177, -1, -1, -1, 882, 145, 13, 7, -1, 87, 129, 141, 14, 1803, 211, 37, -1, -1, -1, 101345, 198, -1, 254, 289, 304, -1, 96, 14, 2385, 33405, 239, 874, -1, 18, -1, -1, -1, -1, -1, -1, 171, 276, -1, 153, 41, 130, 98, 734, 13, 1889, -1, -1, -1, 47, 90, 2521, -1, -1, 922, -1, 159, 61, 41, 79, 1105, 195, 42, 722, 12, -1, 51, -1, 874, -1, 598, 1373, -1, -1, 840, 75, 2466, -1, 96, 282, 1260, 32, -1, 52, 6, 82, 804, 101, -1, 153, 1830, -1, 43, -1, 209, 26, 193, 494, 1879, 261, 426, -1, 13860, 29, 210, 72, 621, 19754, -1, 683, 1250, 60, -1, 92, -1, 405, 208, 1304, 51, 444, -1, -1, 1680, 132, 298, 68, 13, -1, 168, 96, 31, 63, 97935, 528, 185, -1, 38, 86, 50, 223, 125, 148, 801, 338, 753, 54, 141, 1816, 53, -1, -1, 128, 160, 1830, 257, -1, -1, -1, 8519, 9, 282, -1, 9, 105, 101, -1, -1, -1, 1918, -1, -1, 692, 744, 115, 1083, 4000, 468, 3360, -1, 636, 22751, -1, -1, 273, 183, -1, 160, 278, -1, 236, 706, 121, 5, 1680, -1, 20, 209, 16, -1, 2254, -1, 38, 12, 108, 209, 20, 282, -1

In [261]:
#1f. and the number of characters in article’s paragraphs.
targetParagraphs_char_count = []
for x in range(0, len(raw_instance_train)):
    targetParagraphs_char_count.append(count_array_char(raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_char_count)

[3376, 364, 3421, 8954, 4810, 2990, 4901, 628, 271, 3624, 1975, 1799, 5747, 3968, 2788, 3616, 3763, 1450, 6292, 1263, 5117, 424, 424, 2954, 4520, 1010, 6392, 447, 67, 1747, 2743, 2502, 8611, -1, -1, 2670, 220, 1145, 5544, -1, 848, 2306, 2585, 42, 2892, 1151, 2926, 5406, 3395, 1276, 2236, 1910, 3767, 216, 2368, 1562, 1447, 4636, 2128, 3572, 810, 327, 1244, 1660, 3530, 866, 3561, 7478, -1, -1, 3073, 12485, 524, 1071, 2780, 2252, 1613, 2776, 2849, 2959, 9893, -1, 691, -1, 1018, 5943, 4166, -1, 143, 85, 2679, 1710, 2030, 4188, 843, 8640, 2373, 2947, 430, 5140, 2671, 3759, 12820, 829, 2341, 837, 436, -1, 2493, 1491, 2124, 5959, 2253, -1, 3381, 6929, 6959, 5751, 189, 634, 302, 5443, 2419, 2227, 4569, 41, 1187, 445, 614, 9544, 367, 4903, 5084, 1451, 2865, 493, 1491, 3862, 1176, 2197, 2911, 5460, 2037, 1456, 1822, 5767, 8111, 1542, 1751, 4413, 322, 3739, 401, 1028, 3426, 2361, 1343, 5747, 1617, -1, 3440, 361, 326, 3256, 3286, 3889, -1, 2426, 870, -1, 3391, 3083, 150, 3550, 924, 6649, 2627, 237

In [262]:
#2a. the number of words in post’s title.
postText_word_count = []
for x in range(0, len(raw_instance_train)):
    postText_word_count.append(count_tweet_array_word(raw_instance_train[x]['postText']))
print(postText_word_count)

[13, 15, 13, 11, 16, 12, 18, 11, 15, 14, 10, 15, 12, 14, 10, 21, 9, 12, 7, 15, 14, 8, 11, 8, 10, 15, 10, 11, 14, 13, 15, 5, 16, 18, 7, 11, 8, 19, 23, 20, 9, 15, 12, 16, 10, 13, 14, 10, 19, 12, 11, 14, 9, 9, 10, 11, 11, 14, 10, 18, 14, 8, 12, 12, 17, 15, 10, 15, 17, 16, 11, 5, 14, 10, 20, 9, 10, 9, 8, 15, 6, 10, 7, 22, 12, 16, 12, 17, 12, 11, 13, 9, 15, 10, 9, 12, 11, 11, 13, 10, 10, 13, 18, 12, 16, 9, 20, 16, 13, 12, 21, 11, 11, 10, 8, 15, 12, 16, 7, 15, 15, 9, 18, 11, 13, 12, 6, 13, 9, 12, 19, 12, 13, 13, 7, 15, 14, 21, 18, 11, 15, 14, 15, 11, 16, 15, 13, 13, 13, 12, 11, 12, 12, 13, 17, 13, 11, 12, 12, 15, 13, 9, 18, 11, 19, 19, 14, 13, 17, 23, 12, 8, 19, 15, 19, 10, 15, 6, 10, 17, 8, 18, 16, 15, 11, 20, 17, 12, 14, 8, 7, 9, 12, 17, 13, 16, 17, 12, 16, 7, 7, 12, 13, 10, 10, 11, 16, 17, 12, 13, 6, 17, 20, 13, 14, 11, 13, 9, 9, 13, 15, 14, 22, 10, 12, 17, 10, 13, 20, 15, 17, 9, 11, 11, 18, 17, 12, 9, 12, 11, 13, 15, 10, 10, 13, 15, 17, 10, 15, 15, 6, 11, 11, 7, 17, 16, 18, 10, 13, 7, 21

In [263]:
#2b. the number of words in article’s title.
targetTitle_word_count = []
for x in range(0, len(raw_instance_train)):
    targetTitle_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetTitle']))
print(targetTitle_word_count)

[17, 8, 11, 11, 6, 12, 10, 11, 12, 14, 10, 12, 12, 23, 10, 14, 9, 24, 7, 10, 16, 10, 7, 5, 11, 14, 10, 11, 12, 7, 14, 8, 11, 10, 9, 11, 7, 19, 11, 1, 9, 13, 16, 3, 10, 300, 16, 10, 10, 296, 11, 21, 7, 9, 10, 11, 12, 9, 10, 11, 12, 8, 19, 10, 308, 296, 8, 4, 5, 9, 9, 4, 12, 12, 279, 9, 22, 9, 8, 9, 11, 8, 18, 9, 21, 8, 12, 15, 10, 1, 12, 9, 20, 10, 8, 13, 15, 11, 15, 6, 12, 12, 12, 11, 16, 10, 8, 11, 12, 10, 10, 11, 11, 11, 8, 10, 18, 7, 8, 14, 10, 8, 20, 15, 11, 12, 6, 14, 10, 12, 10, 9, 15, 17, 7, 10, 12, 6, 18, 17, 8, 9, 12, 13, 16, 17, 12, 13, 302, 7, 4, 11, 12, 10, 12, 18, 9, 12, 7, 5, 13, 4, 8, 11, 9, 9, 9, 13, 13, 2, 12, 8, 7, 11, 10, 10, 20, 12, 10, 10, 8, 10, 15, 25, 11, 36, 11, 12, 3, 8, 7, 9, 11, 18, 304, 11, 17, 12, 11, 9, 7, 10, 13, 5, 10, 10, 7, 20, 11, 12, 6, 9, 8, 8, 15, 11, 9, 10, 9, 302, 10, 10, 17, 8, 8, 12, 18, 12, 9, 4, 8, 8, 9, 21, 9, 13, 9, 10, 303, 12, 10, 15, 10, 12, 13, 26, 11, 9, 18, 7, 7, 13, 12, 14, 2, 14, 10, 7, 9, 19, 9, 13, 7, 13, 9, 10, 16, 11, 12, 12, 1

In [264]:
#2c. the number of words in article’s description. 
targetDescription_word_count = []
for x in range(0, len(raw_instance_train)):
    targetDescription_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetDescription']))
print(targetDescription_word_count)

[31, 27, 24, 17, 22, -1, 27, 29, 28, 23, 24, 27, 5, 31, 14, 18, 21, 31, 22, 26, 14, 22, 6, 17, 27, -1, 28, -1, -1, 15, 23, 18, 20, 25, -1, 20, 26, 29, 53, 24, 21, 36, 22, -1, 43, 20, 20, 28, 25, 7, 32, 28, 18, 34, 26, 19, 20, 16, 22, 39, 21, 7, 23, 46, 28, 17, 17, 15, 20, 10, 39, 19, 23, 24, 14, 17, 30, 23, 23, 28, -1, 32, 28, 23, 23, 23, 20, 25, 31, -1, 13, 18, 34, 23, 10, 87, 23, 54, 41, 24, 34, 64, -1, 35, 14, 16, 44, 19, -1, 48, 27, 23, 24, 61, 20, 23, -1, 5, 40, -1, 25, 7, 21, 22, 14, -1, 8, -1, 36, -1, 27, 23, 33, 18, 23, 23, 33, 21, 25, 36, 22, 30, 31, 80, 24, 21, 13, 30, 14, 21, 16, 27, -1, 23, 27, 30, 40, 5, 28, -1, 46, 30, 28, 43, 27, 21, 29, 25, -1, -1, 21, 21, 37, 34, 10, 23, 26, 27, 22, 25, -1, 7, 15, 26, 30, 30, 36, 27, 11, 25, 25, 38, -1, 17, 18, 12, 37, 20, 19, 26, 11, 31, 34, 30, 32, 23, 23, 34, 32, 25, 14, 24, 32, 21, 25, 51, 32, 25, -1, 9, 39, 55, 9, 14, 10, -1, 22, 21, 18, 19, 34, -1, 22, 27, 37, 17, 33, 23, 15, 29, 23, 59, -1, 28, 24, 32, 45, 13, 17, 24, 18, 18, 23

In [265]:
#2d. the number of words in article’s keywords.
targetKeywords_word_count = []
for x in range(0, len(raw_instance_train)):
    targetKeywords_word_count.append(count_tweet_string_word(raw_instance_train[x]['targetKeywords']))
print(targetKeywords_word_count)

[13, 19, -1, 18, 11, -1, 10, -1, 20, -1, 7, 5, 15, 4, 6, 4, 9, 15, 8, -1, 7, -1, -1, 8, 31, -1, 25, -1, -1, -1, -1, -1, 10, 11, -1, 9, 26, 3, -1, -1, 10, 3, -1, -1, -1, 15, -1, 24, 11, 14, -1, 17, -1, 40, 2, -1, -1, 7, -1, 12, 8, 12, 9, -1, 35, 16, 19, 6, 6, 16, -1, 21, 7, -1, 10, 13, 17, 23, 14, 11, 13, -1, 13, 12, 6, -1, 18, 20, 1, -1, -1, -1, 11, 14, -1, 4, -1, -1, 50, 9, 3, -1, -1, -1, 6, -1, 22, 4, -1, -1, 8, 4, 7, -1, 11, 9, -1, -1, 6, -1, -1, 3, -1, -1, 9, -1, 7, -1, 3, -1, 18, 13, 10, 4, 18, 6, 5, -1, 4, 4, 10, 22, 12, -1, 16, 6, 10, -1, 15, 9, 4, 14, -1, 6, 32, 12, -1, 15, -1, -1, -1, 28, -1, -1, -1, -1, 21, 10, 13, -1, 12, 10, -1, -1, -1, 19, 9, -1, -1, 18, -1, 6, 23, 19, -1, 5, 10, 17, 4, 10, 21, -1, -1, 3, 11, 4, -1, 12, -1, -1, 25, 48, 11, 1, 14, 5, -1, 13, 28, 10, 8, -1, 14, -1, 20, -1, -1, 16, 27, 14, 11, -1, 8, 18, 8, -1, 13, 8, 10, -1, 8, -1, 12, 13, 18, 3, -1, 19, 17, 5, 9, -1, -1, 10, 8, 17, -1, 30, 3, 9, -1, 9, -1, 3, -1, 8, 9, 9, 6, -1, -1, 10, 13, 20, 40, -1, 10, 

In [ ]:
#2e. the number of words in article’s captions.
targetCaptions_word_count = []
for x in range(0, len(raw_instance_train)):
    targetCaptions_word_count.append(count_tweet_array_word(raw_instance_train[x]['targetCaptions']))
print(targetCaptions_word_count)

[472, 248, 77, 191, 37, -1, -1, -1, 95, 23, 2, 1, -1, 4, 23, 22, 2, 333, 38, 9, -1, -1, -1, 12910, 37, -1, 47, 61, 80, -1, 18, 3, 454, 6063, 48, 180, -1, 1, -1, -1, -1, -1, -1, -1, 32, 50, -1, 30, 9, 28, 19, 136, 2, 250, -1, -1, -1, 13, 13, 523, -1, -1, 174, -1, 31, 13, 9, 15, 211, 34, 6, 153, 2, -1, 10, -1, 182, -1, 121, 267, -1, -1, 170, 12, 506, -1, 18, 60, 195, 6, -1, 10, 1, 15, 153, 4, -1, 31, 232, -1, 2, -1, 39, 7, 33, 96, 248, 46, 97, -1, 2576, 5, 35, 12, 124, 3835, -1, 118, 193, 14, -1, 27, -1, 85, 16, 305, 7, 90, -1, -1, 198, 22, 57, 3, 2, -1, 39, 20, 2, 5, 12285, 111, 40, -1, 6, 18, 6, 43, 26, 32, 179, 61, 152, 8, 29, 354, 9, -1, -1, 25, 23, 232, 48, -1, -1, -1, 1671, 2, 51, -1, 2, 12, 24, -1, -1, -1, 400, -1, -1, 137, 122, 22, 163, 798, 88, 688, -1, 128, 4255, -1, -1, 55, 42, -1, 32, 58, -1, 48, 136, 19, 1, 201, -1, 3, 43, 1, -1, 464, -1, 6, 2, 17, 46, 4, 60, -1, 5, -1, -1, 49, 41, 10, 149, 80, 180, 62, 620, 8, 18, 36, 21, 100, 4, 262, 205, -1, 41, 98, 88, -1, -1, 5, -1, 409

In [ ]:
#2f. and the number of words in article’s paragraphs.
targetParagraphs_word_count = []
for x in range(0, len(raw_instance_train)):
    targetParagraphs_word_count.append(count_tweet_array_word(raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_word_count)

In [ ]:
#3a. the number of common words between article keywords and post’s title.
postText_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    postText_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['postText']))
print(postText_word_frequency)

In [ ]:
#3b. the number of common words between article keywords and article description.
targetDescription_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetDescription_word_frequency.append(keyWordFrequency_string_(keyList, raw_instance_train[x]['targetDescription']))
print(targetDescription_word_frequency)

In [ ]:
#3c. the number of common words between article keywords and article captions.
targetCaptions_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetCaptions_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['targetCaptions']))
print(targetCaptions_word_frequency)

In [ ]:
#3d. the number of common words between article keywords and article paragraphs.
targetParagraphs_word_frequency = []
for x in range(0, len(raw_instance_train)):
    keyList = keyWord_parse_(raw_instance_train[x]['targetKeywords'])
    targetParagraphs_word_frequency.append(keyWordFrequency_array_(keyList, raw_instance_train[x]['targetParagraphs']))
print(targetParagraphs_word_frequency)

In [ ]:
#4a. formal words ratio
formal_word_ratio = []
for x in range(0, len(raw_instance_train)):
    formal_word_ratio.append(count_formal_ratio(raw_instance_train[x]['postText']))
print(formal_word_ratio)

In [ ]:
#5a. tags amount
tags_amount = []
for x in range(len(raw_instance_train)):
    tags_amount.append(tagFrequency_array(tags_parse("#,@"), paragraphs_tags_parse(raw_instance_train[x]['postText'])))
print(tags_amount)

In [246]:
#Whether 'target Title' Start with 5W1H.
wh_start = []
for x in range(0, len(raw_instance_train)):
    wh_start.append(word_tokenize(raw_instance_train[x]['targetTitle'].lower())[0] in ['who', 'when', 'where', 'how', 'why','what'])
#Ethan's edit, I delete the [0] after raw_instance_train[x]['targetTitle']. It's a string already
wh_sample = lambda x: word_tokenize('What is it'.lower())[0] in ['who', 'when', 'where', 'how', 'why','what']
len(wh_start)

17581

In [247]:
#Foward reference: Whether 'target Title' has Demonstratives(this, that, those, these) 
hasDemonstrative = []
for y in range(0, len(raw_instance_train)):
    hasDemonstrative.append(sum(map(lambda x: str(x) in ['this', 'that', 'those','these'], 
                                     word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [248]:
#Foward reference: Whether 'target Title' has definite article('the')
hasDefinite = []
for y in range(0, len(raw_instance_train)):
    hasDefinite.append(sum(map(lambda x: str(x) == 'the', word_tokenize(raw_instance_train[y]
                                                                         ['targetTitle'].lower()))) > 0)


In [249]:
#Foward reference: Whether 'target Title' has 3rd person pronoun('he','she','his','her','him')
hasThirdPronoun = []
for y in range(0, len(raw_instance_train)):
    hasThirdPronoun.append(sum(map(lambda x: str(x) in ['he','she','his','her','him'], word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0)


In [160]:
#Foward reference: Whether 'target Title' Start with adverbs.
startsAdverb = []
for x in range(0, len(raw_instance_train)):
    startsAdverb.append(pos_tag(word_tokenize(raw_instance_train[x]['targetTitle']))[0][1] == 'ADV')
# print(pos_tag(word_tokenize(raw_instance_train[0]['targetTitle']))[0][1])
len(startsAdverb)

17581

In [161]:
countLongestWord = []

for x in range(0, len(raw_instance_train)):
    countLongestWord.append(len(max(raw_instance_train[x]['targetTitle'].split(), key=len)))
countLongestWord

[10,
 8,
 9,
 6,
 7,
 8,
 11,
 12,
 9,
 8,
 10,
 9,
 12,
 13,
 8,
 8,
 11,
 9,
 12,
 10,
 11,
 7,
 9,
 11,
 10,
 6,
 9,
 7,
 6,
 7,
 9,
 6,
 10,
 7,
 13,
 7,
 9,
 8,
 10,
 4,
 12,
 8,
 8,
 9,
 10,
 18,
 12,
 9,
 11,
 18,
 9,
 13,
 10,
 12,
 8,
 9,
 10,
 7,
 13,
 10,
 10,
 8,
 9,
 7,
 18,
 18,
 14,
 4,
 7,
 7,
 9,
 8,
 11,
 8,
 13,
 7,
 11,
 11,
 8,
 5,
 10,
 8,
 10,
 11,
 12,
 7,
 13,
 9,
 9,
 6,
 8,
 7,
 12,
 9,
 12,
 8,
 8,
 8,
 9,
 10,
 10,
 9,
 9,
 10,
 8,
 9,
 7,
 7,
 11,
 10,
 11,
 11,
 9,
 9,
 12,
 9,
 15,
 7,
 12,
 9,
 11,
 10,
 8,
 9,
 8,
 9,
 14,
 10,
 14,
 8,
 9,
 13,
 9,
 8,
 11,
 10,
 9,
 12,
 10,
 11,
 11,
 9,
 11,
 11,
 11,
 10,
 7,
 9,
 18,
 8,
 8,
 8,
 11,
 10,
 9,
 11,
 9,
 12,
 8,
 18,
 11,
 8,
 9,
 7,
 8,
 8,
 8,
 10,
 11,
 7,
 11,
 15,
 6,
 10,
 7,
 11,
 8,
 10,
 10,
 7,
 11,
 9,
 9,
 10,
 10,
 12,
 12,
 10,
 9,
 9,
 5,
 11,
 9,
 10,
 18,
 10,
 10,
 8,
 8,
 11,
 7,
 9,
 9,
 13,
 11,
 10,
 10,
 12,
 6,
 11,
 6,
 17,
 10,
 7,
 9,
 11,
 12,
 10,
 7,
 18,
 9,
 7,
 12,


In [162]:
averageLengthWords = []

for x in range(0, len(raw_instance_train)):
    averageLengthWords.append(sum(len(word) for word in raw_instance_train[x]['targetTitle'].split()) / len(raw_instance_train[x]['targetTitle'].split()))
averageLengthWords

[5.117647058823529,
 5.875,
 5.7,
 4.7272727272727275,
 4.833333333333333,
 4.909090909090909,
 6.7,
 5.090909090909091,
 4.0,
 4.642857142857143,
 6.2,
 5.166666666666667,
 5.5,
 5.695652173913044,
 5.7,
 4.071428571428571,
 6.555555555555555,
 4.916666666666667,
 6.0,
 4.7,
 5.333333333333333,
 4.8,
 5.625,
 5.8,
 5.2727272727272725,
 3.642857142857143,
 6.875,
 4.0,
 3.8,
 4.857142857142857,
 4.571428571428571,
 4.25,
 6.090909090909091,
 3.909090909090909,
 4.818181818181818,
 4.818181818181818,
 6.142857142857143,
 4.947368421052632,
 4.818181818181818,
 4.0,
 4.555555555555555,
 5.076923076923077,
 4.5,
 6.25,
 6.2,
 5.4366666666666665,
 5.1875,
 4.888888888888889,
 5.1,
 5.378378378378378,
 4.7272727272727275,
 6.181818181818182,
 6.857142857142857,
 7.0,
 4.4,
 5.2727272727272725,
 4.6,
 5.0,
 6.2,
 5.5,
 5.166666666666667,
 4.625,
 4.9,
 4.0,
 5.373376623376624,
 5.405405405405405,
 6.625,
 3.0,
 3.6666666666666665,
 4.555555555555555,
 5.666666666666667,
 5.25,
 6.63636363636

In [163]:
#Foward reference: Whether 'target Title' Start with Punctuation.
startsPunctuation = []
for x in range(0, len(raw_instance_train)):
    startsPunctuation.append(raw_instance_train[0]['targetTitle'][0].isdigit())
startsPunctuation

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [201]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [202]:
pos_2gram_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_VBZ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'VBZ'))
pos_2gram_NNP_VBZ

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 8,
 0,
 1,
 0,
 8,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 7,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,


In [203]:
pos_2gram_NN_IN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NN_IN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NN', 'IN'))
pos_2gram_NN_IN

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 2,
 0,
 0,
 12,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 12,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 7,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 14,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 13,
 0,
 1,
 2,
 0,
 1,
 0,
 2,
 0,
 0,


In [204]:
pos_2gram_PRP_VBP = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_PRP_VBP.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'PRP', 'VBP'))
pos_2gram_PRP_VBP

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,


In [205]:
title_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    title_LenCharacter.append(len(raw_instance_train[x]['targetTitle']))
title_LenCharacter

[103,
 54,
 66,
 62,
 34,
 64,
 76,
 67,
 64,
 79,
 71,
 73,
 77,
 153,
 66,
 70,
 67,
 141,
 48,
 56,
 75,
 57,
 52,
 33,
 68,
 64,
 63,
 54,
 47,
 40,
 77,
 41,
 77,
 53,
 63,
 63,
 49,
 112,
 63,
 4,
 49,
 78,
 65,
 29,
 72,
 1930,
 98,
 53,
 61,
 1887,
 63,
 157,
 54,
 56,
 53,
 68,
 55,
 53,
 71,
 64,
 73,
 44,
 118,
 49,
 1962,
 1895,
 61,
 16,
 27,
 49,
 60,
 25,
 83,
 57,
 1766,
 51,
 139,
 63,
 40,
 39,
 83,
 46,
 107,
 43,
 138,
 44,
 71,
 100,
 50,
 6,
 83,
 48,
 124,
 59,
 51,
 75,
 79,
 63,
 73,
 39,
 73,
 61,
 68,
 70,
 68,
 62,
 45,
 57,
 61,
 64,
 76,
 65,
 67,
 57,
 59,
 71,
 100,
 35,
 65,
 73,
 64,
 57,
 109,
 94,
 64,
 54,
 46,
 79,
 76,
 61,
 61,
 65,
 85,
 112,
 50,
 67,
 73,
 56,
 113,
 107,
 54,
 46,
 76,
 90,
 86,
 90,
 58,
 85,
 1921,
 39,
 25,
 68,
 72,
 70,
 79,
 106,
 52,
 77,
 42,
 39,
 80,
 32,
 51,
 62,
 64,
 56,
 51,
 77,
 77,
 13,
 70,
 64,
 37,
 68,
 45,
 73,
 101,
 49,
 62,
 50,
 45,
 71,
 77,
 151,
 67,
 201,
 74,
 71,
 23,
 60,
 30,
 61,
 51,
 103,

In [206]:
keyword_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    keyword_LenCharacter.append(len(raw_instance_train[x]['targetKeywords']))
keyword_LenCharacter

[78,
 129,
 0,
 132,
 70,
 0,
 66,
 0,
 161,
 0,
 47,
 43,
 108,
 21,
 86,
 19,
 55,
 94,
 58,
 0,
 45,
 0,
 0,
 70,
 242,
 0,
 191,
 0,
 0,
 0,
 0,
 0,
 122,
 60,
 0,
 66,
 194,
 32,
 0,
 0,
 71,
 29,
 0,
 0,
 0,
 109,
 0,
 161,
 92,
 108,
 0,
 132,
 0,
 330,
 13,
 0,
 0,
 77,
 0,
 88,
 61,
 76,
 63,
 0,
 249,
 117,
 117,
 43,
 34,
 107,
 0,
 165,
 83,
 0,
 88,
 96,
 114,
 156,
 94,
 61,
 107,
 0,
 88,
 86,
 50,
 0,
 142,
 139,
 4,
 0,
 0,
 0,
 86,
 102,
 0,
 26,
 0,
 0,
 363,
 80,
 23,
 0,
 0,
 0,
 43,
 0,
 158,
 29,
 0,
 0,
 69,
 31,
 73,
 0,
 76,
 58,
 0,
 0,
 43,
 0,
 0,
 30,
 0,
 0,
 102,
 0,
 64,
 0,
 18,
 0,
 152,
 108,
 81,
 21,
 118,
 43,
 32,
 0,
 27,
 21,
 79,
 138,
 103,
 0,
 116,
 50,
 103,
 0,
 102,
 59,
 29,
 106,
 0,
 76,
 238,
 77,
 0,
 108,
 0,
 0,
 0,
 217,
 0,
 0,
 0,
 0,
 147,
 60,
 93,
 0,
 81,
 112,
 0,
 0,
 0,
 128,
 52,
 0,
 0,
 121,
 0,
 45,
 133,
 127,
 0,
 31,
 70,
 122,
 30,
 70,
 116,
 0,
 0,
 17,
 86,
 45,
 0,
 93,
 0,
 0,
 160,
 350,
 78,
 6,
 103,
 43,

In [207]:
caption_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    caption_LenCharacter.append(len(raw_instance_train[x]['targetCaptions']))
caption_LenCharacter

[10,
 36,
 3,
 4,
 3,
 0,
 0,
 0,
 16,
 1,
 1,
 1,
 0,
 4,
 2,
 1,
 1,
 23,
 2,
 1,
 0,
 0,
 0,
 525,
 2,
 0,
 6,
 4,
 8,
 0,
 2,
 1,
 31,
 299,
 14,
 20,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 5,
 4,
 3,
 1,
 4,
 1,
 36,
 0,
 0,
 0,
 1,
 1,
 31,
 0,
 0,
 10,
 0,
 3,
 1,
 4,
 5,
 27,
 7,
 1,
 10,
 1,
 0,
 1,
 0,
 8,
 0,
 12,
 24,
 0,
 0,
 10,
 4,
 24,
 0,
 1,
 2,
 26,
 2,
 0,
 2,
 1,
 1,
 13,
 4,
 0,
 3,
 34,
 0,
 2,
 0,
 2,
 1,
 1,
 13,
 36,
 10,
 4,
 0,
 122,
 1,
 2,
 12,
 13,
 209,
 0,
 26,
 26,
 1,
 0,
 1,
 0,
 13,
 5,
 27,
 1,
 4,
 0,
 0,
 31,
 2,
 6,
 3,
 1,
 0,
 1,
 1,
 1,
 3,
 480,
 7,
 5,
 0,
 2,
 1,
 4,
 1,
 1,
 6,
 74,
 7,
 5,
 1,
 1,
 12,
 1,
 0,
 0,
 11,
 5,
 34,
 3,
 0,
 0,
 0,
 27,
 1,
 1,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 16,
 0,
 0,
 8,
 25,
 1,
 21,
 36,
 13,
 42,
 0,
 13,
 203,
 0,
 0,
 2,
 4,
 0,
 2,
 8,
 0,
 2,
 13,
 1,
 1,
 31,
 0,
 1,
 1,
 1,
 0,
 30,
 0,
 2,
 1,
 7,
 5,
 1,
 2,
 0,
 1,
 0,
 0,
 2,
 9,
 1,
 4,
 7,
 74,
 12,
 18,
 1,
 1,
 1,
 1,
 4,
 2,
 14,
 30,

In [208]:
description_LenCharacter = []
for x in range(0, len(raw_instance_train)):
    description_LenCharacter.append(len(raw_instance_train[x]['targetDescription']))
description_LenCharacter

[184,
 166,
 153,
 114,
 125,
 0,
 154,
 171,
 140,
 157,
 134,
 160,
 36,
 187,
 76,
 100,
 128,
 192,
 129,
 133,
 81,
 127,
 43,
 91,
 147,
 0,
 173,
 0,
 0,
 106,
 146,
 107,
 138,
 161,
 0,
 125,
 152,
 173,
 301,
 156,
 122,
 183,
 140,
 0,
 257,
 105,
 123,
 172,
 138,
 37,
 195,
 180,
 117,
 189,
 166,
 131,
 127,
 88,
 112,
 231,
 175,
 34,
 148,
 302,
 150,
 85,
 125,
 90,
 111,
 60,
 248,
 130,
 146,
 175,
 84,
 101,
 175,
 156,
 135,
 151,
 0,
 204,
 184,
 147,
 158,
 127,
 112,
 163,
 173,
 0,
 112,
 104,
 179,
 134,
 59,
 481,
 133,
 309,
 208,
 153,
 222,
 301,
 0,
 209,
 90,
 96,
 255,
 111,
 0,
 301,
 155,
 136,
 152,
 359,
 147,
 143,
 0,
 29,
 225,
 0,
 132,
 36,
 132,
 152,
 79,
 0,
 51,
 0,
 232,
 0,
 186,
 156,
 191,
 113,
 123,
 124,
 194,
 134,
 145,
 225,
 152,
 151,
 164,
 442,
 156,
 115,
 85,
 167,
 78,
 130,
 89,
 161,
 0,
 138,
 155,
 211,
 247,
 36,
 198,
 0,
 269,
 155,
 143,
 261,
 148,
 124,
 170,
 146,
 0,
 0,
 125,
 121,
 186,
 212,
 56,
 129,
 170,


In [209]:
title_keyword_CharacterDiff = list((map(operator.sub, title_LenCharacter, keyword_LenCharacter)))
title_keyword_CharacterDiff

[25,
 -75,
 66,
 -70,
 -36,
 64,
 10,
 67,
 -97,
 79,
 24,
 30,
 -31,
 132,
 -20,
 51,
 12,
 47,
 -10,
 56,
 30,
 57,
 52,
 -37,
 -174,
 64,
 -128,
 54,
 47,
 40,
 77,
 41,
 -45,
 -7,
 63,
 -3,
 -145,
 80,
 63,
 4,
 -22,
 49,
 65,
 29,
 72,
 1821,
 98,
 -108,
 -31,
 1779,
 63,
 25,
 54,
 -274,
 40,
 68,
 55,
 -24,
 71,
 -24,
 12,
 -32,
 55,
 49,
 1713,
 1778,
 -56,
 -27,
 -7,
 -58,
 60,
 -140,
 0,
 57,
 1678,
 -45,
 25,
 -93,
 -54,
 -22,
 -24,
 46,
 19,
 -43,
 88,
 44,
 -71,
 -39,
 46,
 6,
 83,
 48,
 38,
 -43,
 51,
 49,
 79,
 63,
 -290,
 -41,
 50,
 61,
 68,
 70,
 25,
 62,
 -113,
 28,
 61,
 64,
 7,
 34,
 -6,
 57,
 -17,
 13,
 100,
 35,
 22,
 73,
 64,
 27,
 109,
 94,
 -38,
 54,
 -18,
 79,
 58,
 61,
 -91,
 -43,
 4,
 91,
 -68,
 24,
 41,
 56,
 86,
 86,
 -25,
 -92,
 -27,
 90,
 -30,
 40,
 -45,
 85,
 1819,
 -20,
 -4,
 -38,
 72,
 -6,
 -159,
 29,
 52,
 -31,
 42,
 39,
 80,
 -185,
 51,
 62,
 64,
 56,
 -96,
 17,
 -16,
 13,
 -11,
 -48,
 37,
 68,
 45,
 -55,
 49,
 49,
 62,
 -71,
 45,
 26,
 -56,
 24,
 6

In [210]:
title_caption_CharacterDiff = list((map(operator.sub, title_LenCharacter, caption_LenCharacter)))
title_caption_CharacterDiff

[93,
 18,
 63,
 58,
 31,
 64,
 76,
 67,
 48,
 78,
 70,
 72,
 77,
 149,
 64,
 69,
 66,
 118,
 46,
 55,
 75,
 57,
 52,
 -492,
 66,
 64,
 57,
 50,
 39,
 40,
 75,
 40,
 46,
 -246,
 49,
 43,
 49,
 111,
 63,
 4,
 49,
 78,
 65,
 29,
 71,
 1928,
 98,
 48,
 57,
 1884,
 62,
 153,
 53,
 20,
 53,
 68,
 55,
 52,
 70,
 33,
 73,
 44,
 108,
 49,
 1959,
 1894,
 57,
 11,
 0,
 42,
 59,
 15,
 82,
 57,
 1765,
 51,
 131,
 63,
 28,
 15,
 83,
 46,
 97,
 39,
 114,
 44,
 70,
 98,
 24,
 4,
 83,
 46,
 123,
 58,
 38,
 71,
 79,
 60,
 39,
 39,
 71,
 61,
 66,
 69,
 67,
 49,
 9,
 47,
 57,
 64,
 -46,
 64,
 65,
 45,
 46,
 -138,
 100,
 9,
 39,
 72,
 64,
 56,
 109,
 81,
 59,
 27,
 45,
 75,
 76,
 61,
 30,
 63,
 79,
 109,
 49,
 67,
 72,
 55,
 112,
 104,
 -426,
 39,
 71,
 90,
 84,
 89,
 54,
 84,
 1920,
 33,
 -49,
 61,
 67,
 69,
 78,
 94,
 51,
 77,
 42,
 28,
 75,
 -2,
 48,
 62,
 64,
 56,
 24,
 76,
 76,
 13,
 69,
 61,
 34,
 68,
 45,
 73,
 85,
 49,
 62,
 42,
 20,
 70,
 56,
 115,
 54,
 159,
 74,
 58,
 -180,
 60,
 30,
 59,
 47,
 

In [211]:
caption_title_CharacterRatio = list((map(operator.truediv, caption_LenCharacter, title_LenCharacter)))


In [212]:
description_title_CharacterRatio = list((map(operator.truediv, description_LenCharacter, title_LenCharacter)))


In [213]:
pos_3gram_NNP_NNP_VBZ = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_NNP_NNP_VBZ.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNP', 'VBZ'))
len(pos_3gram_NNP_NNP_VBZ)
print(pos_3gram_NNP_NNP_VBZ)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [214]:
pos_3gram_IN_NNP_NNP = []
for x in range(0, len(raw_instance_train)):
    pos_3gram_IN_NNP_NNP.append(pos_3gram_sum(raw_instance_train[x]['targetTitle'], 'IN','NNP', 'NNP'))
len(pos_3gram_IN_NNP_NNP)
print(pos_3gram_IN_NNP_NNP)

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 1, 8, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 1, 1, 4, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 

In [215]:
pos_2gram_IN_NN = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_NN.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'NN'))
pos_2gram_IN_NN

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,


In [216]:
pos_2gram_NNP_NNS = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_NNP_NNS.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'NNP', 'NNS'))
pos_2gram_NNP_NNS

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 4,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [217]:
pos_2gram_IN_JJ = []
for x in range(0, len(raw_instance_train)):
    pos_2gram_IN_JJ.append(pos_2gram_sum(raw_instance_train[x]['targetTitle'], 'IN', 'JJ'))
pos_2gram_IN_JJ

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 0,
 0,
 6,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,


In [250]:
import csv
from itertools import zip_longest

d = [pos_2gram_IN_JJ, pos_2gram_NNP_NNS, pos_2gram_IN_NN, pos_3gram_IN_NNP_NNP, pos_3gram_NNP_NNP_VBZ, 
     description_title_CharacterRatio, caption_title_CharacterRatio, title_caption_CharacterDiff, title_keyword_CharacterDiff, 
     description_LenCharacter, caption_LenCharacter, keyword_LenCharacter, title_LenCharacter, pos_2gram_PRP_VBP, pos_2gram_NN_IN, 
     pos_2gram_NNP_VBZ, startsPunctuation, averageLengthWords, countLongestWord, startsAdverb, hasThirdPronoun, 
     hasDemonstrative, wh_start, pos_3gram_NNP_NNP_NNP, pos_2gram_NNP_NNP, cal_pos_1grams_tag_list, 
     targetTitle_linsear_write_formula, targetTitle_gunning_fog, targetTitle_coleman_liau_index, 
     targetTitleCountWords, truth_label,
     postTitle_char_count, targetTitle_char_count, targetDescription_char_count, targetKeywords_char_count, 
     targetCaptions_char_count, targetParagraphs_char_count, postText_word_count, targetTitle_word_count,
     targetDescription_word_count, targetKeywords_word_count, targetCaptions_word_count, targetParagraphs_word_count,
     postText_word_frequency,targetDescription_word_frequency, targetCaptions_word_frequency, targetParagraphs_word_frequency,
     formal_word_ratio,tags_amount
    ]
export_data = zip_longest(*d, fillvalue = '')
with open('newFeatures_Ethan.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("pos_2gram_IN_JJ",
                   "pos_2gram_NNP_NNS", "pos_2gram_IN_NN", "pos_3gram_IN_NNP_NNP", "pos_3gram_NNP_NNP_VBZ",
                   "description_title_CharacterRatio", "caption_title_CharacterRatio", "title_caption_CharacterDiff", 
                   "title_keyword_CharacterDiff", "description_LenCharacter", "caption_LenCharacter", 
                   "keyword_LenCharacter", "title_LenCharacter", "startsPunctuation", "averageLengthWords", 
                   "countLongestWord", "startsAdverb", "hasThirdPronoun", "hasDemonstrative", "wh_start", 
                   "pos_3gram_NNP_NNP_NNP", "pos_2gram_NNP_NNP", "cal_pos_1grams_tag_list", 
                   "targetTitle_linsear_write_formula", "targetTitle_gunning_fog", "targetTitle_coleman_liau_index", 
                   "targetTitleCountWords","truth_label", 
                   "postTitle_char_count", "targetTitle_char_count","targetDescription_char_count", 
                   "targetKeywords_char_count", "targetCaptions_char_count", "targetParagraphs_char_count", 
                   "postText_word_count", "targetTitle_word_count", "targetDescription_word_count", 
                   "targetKeywords_word_count, targetCaptions_word_count, targetParagraphs_word_count",
                   "postText_word_frequency", "targetDescription_word_frequency", "targetCaptions_word_frequency", 
                   "targetParagraphs_word_frequency", "formal_word_ratio", "tags_amount"))
    wr.writerows(export_data)
myfile.close()